## This Notebook is for geometrics related searches and shows how the requests can be built

### Initialize

In [4]:
import os
import sys
from datetime import datetime

import pandas as pd
import requests

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
import src.esh_objects as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path

base_url = get_base_url()
root_path = get_root_path()
selected = sys.path[0].split(os.sep)[-2]



def execute_search_query(query):
    r = requests.post(f'{base_url}/v1/query/{selected}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[ ['entityDesc', 'position','street', 'houseNumber', 'city', 'postalCode', 'country']]
            df.rename(columns={'entityDesc': 'Description',
                  'position': 'Position', 'street': 'Street', 'houseNumber': 'House Number', 'city': 'City',
                           'postalCode': 'Postal Code', 'country': 'Country'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())


## Search for position Latitude='51.38026158483485'

In [5]:
so = esh.EshObject(
    count=True,
    top=2,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Location'),
            esh.Comparison(
                property=esh.Property(property='position'),
                operator=esh.ComparisonOperator.Search,
                value=esh.Point(
                    coordinates=[51.560760544987865,-0.35568797163571464]))]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))


ESH query statement: /$all?$top=2&$count=true&$apply=filter(Search.search(query='SCOPE:Location AND position:POINT(51.560760544987865 -0.35568797163571464)'))
Total count: 2


,Description,Position,Street,House Number,City,Postal Code,Country
0,Kennedy Cleaning Services Pty Ltd Premises,"{'type': 'Point', 'coordinates': [51.560760544...",Reed,68,London,2900,England
1,Royal Constuction Pty Ltd Premises,"{'type': 'Point', 'coordinates': [51.560760544...",Monaro,121,London,2620,England


## Search for the location coordinates of the person "Alex Kennedy"

In [8]:
so = esh.EshObject(
    count=True,
    top=2,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Location'),
            esh.Comparison(
                property=esh.Property(
                    property=['relPerson', 'person', 'lastName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='Kennedy')),
            esh.Comparison(
                property=esh.Property(
                    property=['relPerson', 'person', 'firstName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='Alex'))
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))


ESH query statement: /$all?$top=2&$count=true&$apply=filter(Search.search(query='SCOPE:Location AND relPerson.person.lastName:Kennedy AND relPerson.person.firstName:Alex'))
Total count: 1


,Description,Position,Street,House Number,City,Postal Code,Country
0,Alex Kennedy Residence,"{'type': 'Point', 'coordinates': [51.324513706...",Ellerston,131,London,2905,England


## Search for exact coordinates 51.560760544987865 -0.35568797163571464

In [9]:
so = esh.EshObject(
    count=True,
    top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Location'),
            esh.Comparison(
                property=esh.Property(property='position'),
                operator=esh.ComparisonOperator.Search,
                value=esh.Point(
                    coordinates=[51.560760544987865, -0.35568797163571464]))]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))

ESH query statement: /$all?$top=1&$count=true&$apply=filter(Search.search(query='SCOPE:Location AND position:POINT(51.560760544987865 -0.35568797163571464)'))
Total count: 2


,Description,Position,Street,House Number,City,Postal Code,Country
0,Kennedy Cleaning Services Pty Ltd Premises,"{'type': 'Point', 'coordinates': [51.560760544...",Reed,68,London,2900,England


## Search for person wihthin a radius of 10 km from coordinates [51.496683944068565, 0.08788500846256075]


In [12]:
so = esh.EshObject(
    count=True,
    top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property=esh.Property(property=['relLocation','location','position']),
                operator=esh.ComparisonOperator.Search,
                value=esh.Point(
                    coordinates=[51.560760544987865, -0.35568797163571464]))]))

print(f'ESH query statement: {so.to_statement()}')
# execute_search_query(so.dict(exclude_none=True))


r = requests.post(f'{base_url}/v1/query/{selected}{TENANT_SUFFIX}/latest', json=[so.dict()])
if r.status_code == 200:
    res = r.json()[0]
    if '@odata.count' in res:
        print(f'Total count: {res["@odata.count"]}')
    if res and 'value' in res and res['value']:
        df = pd.DataFrame.from_dict(res['value'])[['firstName', 'lastName','age']]
        df.rename(columns={'firstName': 'First Name', 'lastName': 'Last Name', 'age':'Age'}, inplace=True)
        display(df)
    else:
        print('No results')
else:
    print(f'Error during search. HTTP status code {r.status_code}')
    print(r.json())

<>:12: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:12: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
C:\Users\I570329\AppData\Local\Temp\ipykernel_35220\1459064029.py:12: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
  coordinates=':WITHIN:'[51.560760544987865, -0.35568797163571464]))]))
C:\Users\I570329\AppData\Local\Temp\ipykernel_35220\1459064029.py:12: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
  coordinates=':WITHIN:'[51.560760544987865, -0.35568797163571464]))]))
C:\Users\I570329\AppData\Local\Temp\ipykernel_35220\1459064029.py:12: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
  coordinates=':WITHIN:'[51.560760544987865, -0.35568797163571464]))]))


TypeError: string indices must be integers